In [1]:
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.models import load_model
import numpy as np
import pickle
import cv2
import os
import matplotlib.pyplot as plt
from sklearn import metrics
import random
from pathlib import PurePosixPath
import pandas as pd

In [2]:
model_path = "output_1/detector.h5"
binarizer_path = "output_1/lb.pickle"
model = load_model(model_path)
lb = pickle.loads(open(binarizer_path, "rb").read())

f = open("output_1/test_paths.txt", "r")
imagePaths = list(f)
imagePaths = [each.strip("\n").replace(",","") for each in imagePaths] # need to fix this shit

f = open("output_1/test_labels.txt", "r")
imageLabs = list(f)
imageLabs = [each.strip("\n").replace(",","") for each in imageLabs] # and this shit

/opt/anaconda3/lib/python3.7/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator LabelBinarizer from version 0.23.2 when using version 0.22.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [3]:
# True labels
y_true = []
for image in imageLabs:
    img = np.array([int(i) for i in image]) # need to fix this lmao so inefficient
    img = img.reshape(1,8)
    index = np.argmax(img, axis=1)
    label = lb.classes_[index][0]
    y_true.append(label)

In [6]:
%%time
# True bounding boxes
# 23s for 1600 values
Annots = pd.read_csv("Annotations_8catags2.csv").drop("Unnamed: 0",axis=1)
def get_bboxes(imgpath):
    p = PurePosixPath(imgpath)
    img = str(p.relative_to("DeepFashion")) # another data mismatch - maybe i'll fix it someday 
    x1,x2,y1,y2 = Annots.loc[Annots['Image'] ==img].values[0][2:] # trace bboxes from Annotations file
    return (x1,x2,y1,y2) # change to (x1,y1,x2,y2) after training the new model

bboxes_true = []
for path in imagePaths:
    x1,y1,x2,y2 = get_bboxes(path)
    bboxes_true.append((x1,y1,x2,y2))

CPU times: user 23.6 s, sys: 92.8 ms, total: 23.7 s
Wall time: 23.8 s


In [ ]:
%%time
# Predicted labels and bounding boxes
# takes 5:30 for 1600 values - no time savings by reading images only with cv2.
bboxes_pred = []
y_pred = []
for img in imagePaths:
    image = load_img(img, target_size=(224, 224))
    image = img_to_array(image) / 255.0
    image = np.expand_dims(image, axis=0)
    (boxPreds, labelPreds) = model.predict(image)
    (x1, y1, x2, y2) = boxPreds.flatten()
    image = cv2.imread(img)
    (h, w) = image.shape[:2]
    # re-scale the bounding box coordinates to fit the original image
    x1 = int(x1 * w)
    y1 = int(y1 * h)
    x2 = int(x2 * w)
    y2 = int(y2 * h)
    bboxes_pred.append((x1, y1, x2, y2))
    index = np.argmax(labelPreds, axis=1)
    label = lb.classes_[index][0]
    y_pred.append(label)

In [ ]:
print(metrics.confusion_matrix(y_true,y_pred,labels=lb.classes_))

In [ ]:
print(metrics.classification_report(y_true, y_pred,labels=lb.classes_))

In [ ]:
def plot_bboxes(imagepath,true_bbox,pred_bbox):
    x1,x2,y1,y2 = true_bbox
    rx1,rx2,ry1,ry2 = pred_bbox
    img2 = cv2.imread(imagepath)
    img2 = cv2.cvtColor(img2, cv2.COLOR_BGR2RGB)
    copy = img2.copy()
    plt.imshow(cv2.rectangle(copy,(x1,y1),(x2,y2),(36,255,12),2)) # actual - green
    plt.imshow(cv2.rectangle(copy,(rx1,ry1),(rx2,ry2),(255,36,12),2)) #predicted - red
    plt.show()

In [ ]:
randomlist = random.sample(range(0, 1600), 25)
for i in (randomlist):
    true_bbox = bboxes_true[i]
    pred_bbox = bboxes_pred[i]
    img = imagePaths[i]
    plot_bboxes(img,true_bbox,pred_bbox)